## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

## Calcolo resa globale

In [3]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0380952380952382)'

## Funzioni di calcolo

In [4]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    return ricetta.reshape(-1, len(produzioni)) * produzioni

In [5]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [6]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum(np.square(tot_consumi-consumi))
    return tot_err

In [7]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [8]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

## Analisi composizione

In [9]:
# Matrice consumi usando ricetta iniziale
cons_fam = calc_mat_consumi(ricette)
pd.DataFrame(cons_fam)

,0,1,2
0,750.0,1260.0,360.0
1,1290.0,3150.0,420.0
2,900.0,1701.0,420.0
3,60.0,189.0,0.0


In [10]:
# Reshape della matrice composizioni (famiglia vs materiale)
compos = composizioni_famiglia.reshape(len(consumi), -1)
pd.DataFrame(compos, columns=['Mat1', 'Mat2', 'Mat3'])

,Mat1,Mat2,Mat3
0,0.58,0.42,0.0
1,1.00,0.00,0.0
2,0.00,1.00,0.0
3,0.00,0.00,1.0


In [11]:
# Devo calcolare quanto materiale per ciascuna ricetta

# Proviamo per prima colonna matrice consumi (quindi prima ricetta)
# Moltiplico la colonna consumi di quella ricetta per le singole colonne della matrice composizione

cons_ricetta0 = np.vstack(cons_fam[:,0])
print(cons_ricetta0)
print('')
cons_materiali_ricetta0 = cons_ricetta0 * compos
print(cons_materiali_ricetta0)
print('')
tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
print(tot_cons_ricetta0)
print('')
print('Composizione ricetta 0')
tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0)
print(tot_perc_ricetta0)

[[ 750.]
 [1290.]
 [ 900.]
 [  60.]]

[[ 435.  315.    0.]
 [1290.    0.    0.]
 [   0.  900.    0.]
 [   0.    0.   60.]]

[1725. 1215.   60.]

Composizione ricetta 0
[0.575 0.405 0.02 ]


In [12]:
# Percentuali di ciascun materiale in una ricetta
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [13]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [14]:
constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if any(mat):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
del constraints[5]

for x in constraints:
    print(x)

bounds = list(( (0, None) for x in range(len(produzioni)*len(consumi)) ))
# bounds[4] = (0, 0.5) #DEBUG

res = minimize(
    calc_error_resa, 
    ricette, 
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'disp': True, 'maxiter':100}
)

print(res)

{'type': 'eq', 'fun': <function calc_err_totali at 0x0000016D436FE5C0>}
{'type': 'ineq', 'fun': <function err_perc_mat at 0x0000016D436FD940>, 'args': (0, 0, 0.58, 0.01)}
{'type': 'ineq', 'fun': <function err_perc_mat at 0x0000016D436FD940>, 'args': (0, 1, 0.396, 0.003)}
{'type': 'ineq', 'fun': <function err_perc_mat at 0x0000016D436FD940>, 'args': (0, 2, 0.024, 0.001)}
{'type': 'ineq', 'fun': <function err_perc_mat at 0x0000016D436FD940>, 'args': (1, 0, 0.625, 0.005)}
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.64746502685752e-16
            Iterations: 28
            Function evaluations: 369
            Gradient evaluations: 28
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.64746502685752e-16
       x: [ 2.024e-01  8.731e-02  2.857e-01  4.874e-01  6.012e-01
            3.751e-01  3.234e-01  3.157e-01  3.676e-01  2.490e-02
            3.392e-02  9.680e-03]
     nit: 28
     jac: [ 1.868e-10 -1.455e

In [15]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [16]:
pd.options.display.float_format = '{:.6f}'.format

In [17]:
print("Percentuali aggiustate (in %)")
pd.DataFrame(
    res.x.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)


,0,1,2
0,20.237896,8.731058,28.567166
1,48.740183,60.121563,37.511361
2,32.341145,31.565242,36.762982
3,2.490300,3.391661,0.968014


In [18]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2
0,607.136877,550.056623,342.805987
1,1462.205482,3787.658459,450.136331
2,970.234337,1988.610255,441.155787
3,74.708997,213.674616,11.616172


In [19]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3
0,1499.999487,5700.000272,3400.000379,299.999784


In [20]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3
0,-0.000513,0.000272,0.000379,-0.000216


In [21]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2
0,1.038095,1.038095,1.038095


In [22]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2
0,-0.000000,-0.000000,-0.000000


In [23]:
print('Percentuali materiali ricetta 0')
print(range_ric_mat[0])
pd.DataFrame([perc_mat(res.x, 0) * 100])

Percentuali materiali ricetta 0
[[0.58 0.01]
 [0.396 0.003]
 [0.024 0.001]]


,0,1,2
0,58.258781,39.342307,2.398913


In [24]:
print('Percentuali materiali ricetta 1')
print(range_ric_mat[1])
pd.DataFrame([perc_mat(res.x, 1) * 100])

Percentuali materiali ricetta 1
[[0.625 0.005]
 [None None]
 [None None]]


,0,1,2
0,62.793445,33.939359,3.267196


In [25]:
print('Percentuali materiali ricetta 2')
print(range_ric_mat[2])
pd.DataFrame([perc_mat(res.x, 2) * 100])

Percentuali materiali ricetta 2
[[0.62 0.01]
 [None None]
 [None None]]


,0,1,2
0,52.095719,46.971791,0.932491
